In [ ]:
!pip list

Package                       Version
----------------------------- -------------------
absl-py                       0.12.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.1.0
appdirs                       1.4.4
argcomplete                   1.12.3
argon2-cffi                   20.1.0
arviz                         0.11.2
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-generator               1.10
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.2.0
audioread                     2.1.9
autograd                      1.3
Babel                         2.9.1
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        4.0.0
blis                          0.4.1
bokeh                         2.3.3
Bottleneck                    1.3.2
branca                        0.4.2
bs4     

In [ ]:
pip install happytransformer

     |████████████████████████████████| 44 kB 1.4 MB/s 
     |████████████████████████████████| 264 kB 9.1 MB/s 
     |████████████████████████████████| 2.6 MB 56.5 MB/s 
     |████████████████████████████████| 1.2 MB 62.5 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 243 kB 79.0 MB/s 
     |████████████████████████████████| 118 kB 89.7 MB/s 
     |████████████████████████████████| 636 kB 62.4 MB/s 
     |████████████████████████████████| 895 kB 65.9 MB/s 
     |████████████████████████████████| 3.3 MB 52.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!nvidia-smi

Tue Aug 17 09:54:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from happytransformer import HappyQuestionAnswering, QATrainArgs, QATestArgs, QAEvalArgs
import torch
import pandas as pd
import shutil
from google.colab import drive
from psutil import virtual_memory


In [ ]:
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Aug 17 09:55:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
deberta_qa = HappyQuestionAnswering("DEBERTA","microsoft/deberta-large")


Downloading:   0%|          | 0.00/475 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-large were not used when initializing DebertaForQuestionAnswering: ['lm_predictions.lm_head.bias', 'config', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'deberta.embeddings.position_embeddings.weight', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForQuestionAnswering were not initialized from the model checkpoint at microsoft/deberta-large and are newly ini

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

08/17/2021 09:57:51 - INFO - happytransformer.happy_transformer -   Using model: cuda


In [ ]:
torch.version.cuda

'10.2'

In [ ]:
deberta_qa.model

DebertaForQuestionAnswering(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0): DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=1024, out_features=3072, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=1024, out_features=1024, bias=False)
              (pos_q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (inter

In [ ]:
deberta_qa.model_name

'microsoft/deberta-large'

In [ ]:
deberta_qa.model_type

'DEBERTA'

In [ ]:
deberta_train_args = QATrainArgs(learning_rate=2e-5, batch_size=1)


In [ ]:
deberta_train_args


QATrainArgs(learning_rate=2e-05, num_train_epochs=3.0, weight_decay=0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, save_preprocessed_data=False, save_preprocessed_data_path='', load_preprocessed_data=False, load_preprocessed_data_path='', batch_size=1)

In [ ]:
pd.read_csv("./drive/MyDrive/Research_Transformer/train/train_qa_10k.csv")

,context,question,answer_text,answer_start
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,269
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,207
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,526
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",166
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,276
...,...,...,...,...
9995,The Oklahoman is Oklahoma City's major daily n...,What is the name of The Oklahoman's website?,NewsOK.com,103
9996,The Oklahoman is Oklahoma City's major daily n...,What is Oklahoma Cities newsweekly?,Oklahoma Gazette,150
9997,The Oklahoman is Oklahoma City's major daily n...,What is Oklahoma Cities daily business newspaper?,The Journal Record,338
9998,There are numerous community and international...,Which side is the Black Chronicles Headquarters?,Eastside,156


In [ ]:
!nvidia-smi

Tue Aug 17 14:21:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    38W / 250W |  11271MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
train_output = deberta_qa.train("./drive/MyDrive/Research_Transformer/train/train_qa_10k.csv",args=deberta_train_args)

08/17/2021 10:01:59 - INFO - happytransformer.happy_transformer -   Preprocessing dataset...
***** Running training *****
  Num examples = 10000
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Step,Training Loss
500,1.997200
1000,1.773900
1500,1.690800
2000,1.550600
2500,1.614300
3000,1.593500
3500,1.542800
4000,1.528200
4500,1.538500
5000,1.465300


Saving model checkpoint to /tmp/tmpn95109pv/checkpoint-500
Configuration saved in /tmp/tmpn95109pv/checkpoint-500/config.json
Model weights saved in /tmp/tmpn95109pv/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /tmp/tmpn95109pv/checkpoint-500/tokenizer_config.json
Special tokens file saved in /tmp/tmpn95109pv/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /tmp/tmpn95109pv/checkpoint-1000
Configuration saved in /tmp/tmpn95109pv/checkpoint-1000/config.json
Model weights saved in /tmp/tmpn95109pv/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /tmp/tmpn95109pv/checkpoint-1000/tokenizer_config.json
Special tokens file saved in /tmp/tmpn95109pv/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to /tmp/tmpn95109pv/checkpoint-1500
Configuration saved in /tmp/tmpn95109pv/checkpoint-1500/config.json
Model weights saved in /tmp/tmpn95109pv/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in /tmp/tmpn95109pv/checkpo

In [ ]:
pd.read_csv('./drive/MyDrive/Research_Transformer/val/val_qa_6k.csv')

,context,question,answer_text,answer_start
0,Another possible cause of diarrhea is irritabl...,What is IBS?,irritable bowel syndrome,38
1,Another possible cause of diarrhea is irritabl...,What symptoms do you have with IBS?,abdominal discomfort relieved by defecation an...,98
2,Another possible cause of diarrhea is irritabl...,How can symptoms be managed with IBS?,"a combination of dietary changes, soluble fibe...",302
3,Poverty is a good indicator of the rate of inf...,What is a good indicator of the rate of infect...,Poverty,0
4,Poverty is a good indicator of the rate of inf...,Why does living in poverty raise your risk of ...,from the conditions under which impoverished p...,143
...,...,...,...,...
5995,"BJTs have three terminals, corresponding to th...",How can one find the amount of collector current?,β (common-emitter current gain) times the base...,945
5996,"BJTs have three terminals, corresponding to th...",What is the usual collector current for small-...,greater than 100,1017
5997,"In a FET, the drain-to-source current flows vi...",What determines the conductivity in a FET?,electric field that is produced when a voltage...,156
5998,"In a FET, the drain-to-source current flows vi...",What controls the current between the drain an...,the voltage applied between the gate and source,325


In [ ]:
deberta_eval_args = QAEvalArgs()

In [ ]:
deberta_eval_args

QAEvalArgs(batch_size=1, save_preprocessed_data=False, save_preprocessed_data_path='', load_preprocessed_data=False, load_preprocessed_data_path='')

In [ ]:
eval_output = deberta_qa.eval("./drive/MyDrive/Research_Transformer/val/val_qa_6k.csv", args=deberta_eval_args)


PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 6000
  Batch size = 1


In [ ]:
eval_output


EvalResult(loss=2.3976423740386963)

In [ ]:
!mkdir ./drive/MyDrive/Research_Transformer/models

mkdir: cannot create directory ‘./drive/MyDrive/Research_Transformer/models’: File exists


In [ ]:
!mkdir ./drive/MyDrive/Research_Transformer/models/deberta

In [ ]:
!mkdir ./drive/MyDrive/Research_Transformer/models/deberta/model

In [ ]:
!ls ./drive/MyDrive/Research_Transformer/

models	squad  test  train  val


In [ ]:
!ls ./drive/MyDrive/Research_Transformer/models/

albert	deberta  roberta


In [ ]:
!ls ./drive/MyDrive/Research_Transformer/models/deberta/

model


In [ ]:
deberta_eval_loss = {'eval_loss' : [eval_output.loss]}


In [ ]:
deberta_eval_loss


{'eval_loss': [2.3976423740386963]}

In [ ]:
deberta_eval_loss_df = pd.DataFrame(deberta_eval_loss)


In [ ]:
deberta_eval_loss_df


,eval_loss
0,2.397642


In [ ]:
deberta_eval_loss_df.to_csv(r'./drive/MyDrive/Research_Transformer/models/deberta/deberta_eval_loss.csv', index=False)


In [ ]:
pd.read_csv('./drive/MyDrive/Research_Transformer/models/deberta/deberta_eval_loss.csv')


,eval_loss
0,2.397642


In [ ]:
deberta_sample_result = deberta_qa.answer_question("Macedonia was an ancient kingdom on the periphery of Archaic and Classical Greece, and later the dominant state of Hellenistic Greece. The kingdom was founded and initially ruled by the Argead dynasty, followed by the Antipatrid and Antigonid dynasties. Home to the ancient Macedonians, it originated on the northeastern part of the Greek peninsula. Before the 4th century BC, it was a small kingdom outside of the area dominated by the city-states of Athens, Sparta and Thebes, and briefly subordinate to Achaemenid Persia.",
                         "Who ruled Macedonia?")


In [ ]:
deberta_sample_result


[QuestionAnsweringResult(answer=' the Argead dynasty', score=0.9661486148834229, start=181, end=200)]

In [ ]:
pd.read_csv('./drive/MyDrive/Research_Transformer/test/test_qa_5k.csv')


,context,question,answer_text,answer_start
0,Required attendance at school is 10 years for ...,How long are females required to go to school?,11 years,56
1,Required attendance at school is 10 years for ...,What was the literacy rate on Tuvalu in 2002?,99.0%,112
2,Required attendance at school is 10 years for ...,How many students were in Tuvalu schools in 2010?,"1,918",146
3,Required attendance at school is 10 years for ...,What si the teacher-student ratio for Tuvalu s...,1:18,292
4,Community Training Centres (CTCs) have been es...,What type of school has Tuvalu set up on each ...,Community Training Centres,0
...,...,...,...,...
4995,"In the years that followed, Eisenhower increas...",How many soldiers did Eisenhower ultimately se...,900,106
4996,"In the years that followed, Eisenhower increas...",How many days did the President of South Vietn...,ten,313
4997,"In the years that followed, Eisenhower increas...",Where was a parade held for the President of S...,New York City,416
4998,"In the years that followed, Eisenhower increas...",Who was Secretary of State in 1957?,John Foster Dulles,497


In [ ]:
pd.read_csv('./drive/MyDrive/Research_Transformer/test/test_qa_without_answer_5k.csv')


,context,question
0,Required attendance at school is 10 years for ...,How long are females required to go to school?
1,Required attendance at school is 10 years for ...,What was the literacy rate on Tuvalu in 2002?
2,Required attendance at school is 10 years for ...,How many students were in Tuvalu schools in 2010?
3,Required attendance at school is 10 years for ...,What si the teacher-student ratio for Tuvalu s...
4,Community Training Centres (CTCs) have been es...,What type of school has Tuvalu set up on each ...
...,...,...
4995,"In the years that followed, Eisenhower increas...",How many soldiers did Eisenhower ultimately se...
4996,"In the years that followed, Eisenhower increas...",How many days did the President of South Vietn...
4997,"In the years that followed, Eisenhower increas...",Where was a parade held for the President of S...
4998,"In the years that followed, Eisenhower increas...",Who was Secretary of State in 1957?


In [ ]:
deberta_test_args = QATestArgs()


In [ ]:
deberta_test_args


QATestArgs(save_preprocessed_data=False, save_preprocessed_data_path='', load_preprocessed_data=False, load_preprocessed_data_path='')

In [ ]:
deberta_result = deberta_qa.test('./drive/MyDrive/Research_Transformer/test/test_qa_without_answer_5k.csv', args=deberta_test_args)


5000it [03:49, 21.74it/s]


In [ ]:
deberta_result


[QuestionAnsweringResult(answer=' 11 years', score=0.9988104701042175, start=55, end=64),
 QuestionAnsweringResult(answer=' 99.0%', score=0.9964561462402344, start=111, end=117),
 QuestionAnsweringResult(answer=' 1,918', score=0.9997463226318359, start=145, end=151),
 QuestionAnsweringResult(answer=' 1:18', score=0.1304776668548584, start=291, end=296),
 QuestionAnsweringResult(answer='Community Training Centres', score=0.39153435826301575, start=0, end=26),
 QuestionAnsweringResult(answer=' vocational training', score=0.9993812441825867, start=114, end=134),
 QuestionAnsweringResult(answer=' secondary education', score=0.9995689988136292, start=230, end=250),
 QuestionAnsweringResult(answer=' Adults', score=0.9999604821205139, start=496, end=503),
 QuestionAnsweringResult(answer=' vocational training', score=0.6369720697402954, start=114, end=134),
 QuestionAnsweringResult(answer=' plants and trees', score=0.9897570013999939, start=40, end=57),
 QuestionAnsweringResult(answer=' screw 

In [ ]:
deberta_result[4997].answer

' New York City'

In [ ]:
deberta_result_df = pd.DataFrame([vars(a) for a in deberta_result])


In [ ]:
deberta_result_df


,answer,score,start,end
0,11 years,0.998810,55,64
1,99.0%,0.996456,111,117
2,"1,918",0.999746,145,151
3,1:18,0.130478,291,296
4,Community Training Centres,0.391534,0,26
...,...,...,...,...
4995,900,0.935915,105,109
4996,ten,0.989804,312,316
4997,New York City,0.991784,415,429
4998,John Foster Dulles,0.999975,496,515


In [ ]:
deberta_result_df.to_csv(r'./drive/MyDrive/Research_Transformer/models/deberta/deberta_result.csv', index=False)


In [ ]:
pd.read_csv('./drive/MyDrive/Research_Transformer/models/deberta/deberta_result.csv')


,answer,score,start,end
0,11 years,0.998810,55,64
1,99.0%,0.996456,111,117
2,"1,918",0.999746,145,151
3,1:18,0.130478,291,296
4,Community Training Centres,0.391534,0,26
...,...,...,...,...
4995,900,0.935915,105,109
4996,ten,0.989804,312,316
4997,New York City,0.991784,415,429
4998,John Foster Dulles,0.999975,496,515


In [ ]:
pd.read_csv('./drive/MyDrive/Research_Transformer/test/test_qa_5k.csv')


,context,question,answer_text,answer_start
0,Required attendance at school is 10 years for ...,How long are females required to go to school?,11 years,56
1,Required attendance at school is 10 years for ...,What was the literacy rate on Tuvalu in 2002?,99.0%,112
2,Required attendance at school is 10 years for ...,How many students were in Tuvalu schools in 2010?,"1,918",146
3,Required attendance at school is 10 years for ...,What si the teacher-student ratio for Tuvalu s...,1:18,292
4,Community Training Centres (CTCs) have been es...,What type of school has Tuvalu set up on each ...,Community Training Centres,0
...,...,...,...,...
4995,"In the years that followed, Eisenhower increas...",How many soldiers did Eisenhower ultimately se...,900,106
4996,"In the years that followed, Eisenhower increas...",How many days did the President of South Vietn...,ten,313
4997,"In the years that followed, Eisenhower increas...",Where was a parade held for the President of S...,New York City,416
4998,"In the years that followed, Eisenhower increas...",Who was Secretary of State in 1957?,John Foster Dulles,497


In [ ]:
deberta_qa.save('./drive/MyDrive/Research_Transformer/models/deberta/model')

Configuration saved in ./drive/MyDrive/Research_Transformer/models/deberta/model/config.json
Model weights saved in ./drive/MyDrive/Research_Transformer/models/deberta/model/pytorch_model.bin
tokenizer config file saved in ./drive/MyDrive/Research_Transformer/models/deberta/model/tokenizer_config.json
Special tokens file saved in ./drive/MyDrive/Research_Transformer/models/deberta/model/special_tokens_map.json


In [ ]:
!ls -lhac './drive/MyDrive/Research_Transformer/models/deberta/model'


total 1.6G
-rw------- 1 root root  784 Aug 17 14:49 config.json
-rw------- 1 root root 446K Aug 17 14:49 merges.txt
-rw------- 1 root root 1.6G Aug 17 14:49 pytorch_model.bin
-rw------- 1 root root  778 Aug 17 14:49 special_tokens_map.json
-rw------- 1 root root 1.2K Aug 17 14:49 tokenizer_config.json
-rw------- 1 root root 1.3M Aug 17 14:49 tokenizer.json
-rw------- 1 root root 780K Aug 17 14:49 vocab.json
